In [5]:
from keras.datasets import mnist
import pandas as pd
import numpy as np
import math
import random
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
import seaborn as sbn
import sklearn.metrics as sm
import pickle
from sklearn.naive_bayes import GaussianNB as nb
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.linear_model import LogisticRegression as lr
import warnings
import os
import cv2
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential
from keras.callbacks import *
from keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import TensorBoard
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau
from tqdm import tqdm
from skimage.filters import gabor
from sklearn.preprocessing import StandardScaler

# Tắt cảnh báo
warnings.filterwarnings("ignore")

In [6]:
def compute_gabor_features(image, orientations, scales):
    gabor_features = []
    for theta in orientations:
        for sigma in scales:
            real, imag = gabor(image, frequency=0.6, theta=theta, sigma_x=sigma, sigma_y=sigma)
            gabor_features.append(np.mean(real))
            gabor_features.append(np.std(real))
    return gabor_features

In [7]:
def compute_hsv_features(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h_mean = np.mean(hsv_image[:, :, 0])
    h_std = np.std(hsv_image[:, :, 0])
    s_mean = np.mean(hsv_image[:, :, 1])
    s_std = np.std(hsv_image[:, :, 1])
    v_mean = np.mean(hsv_image[:, :, 2])
    v_std = np.std(hsv_image[:, :, 2])
    return [h_mean, h_std, s_mean, s_std, v_mean, v_std]

In [8]:
def extract_features(image):
    image = cv2.resize(image, (128, 128))
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Define Gabor filter parameters
    orientations = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    scales = [1, 2]
    
    # Compute Gabor features
    gabor_features = compute_gabor_features(gray_image, orientations, scales)
    
    # Compute HSV color features
    hsv_features = compute_hsv_features(image)
    
    # Combine features into a single feature vector
    feature_vector = gabor_features + hsv_features
    
    return feature_vector

In [9]:
train_X = []
train_y = []
with tqdm(total=len(os.listdir('./Dataset/train/'))) as pbar:
    for i in os.listdir('./Dataset/train/'):
        img = cv2.imread(f'./Dataset/train/{i}')
        train_y.append(i.split('.')[0].split('_')[1])
        train_X.append(extract_features(img))
        pbar.update(1)
train_X = np.array(train_X)
train_y = np.array(train_y)

100%|████████████████████████████████████████████████████████████████████████| 240000/240000 [2:15:37<00:00, 29.49it/s]


In [10]:
test_X = []
test_y = []
with tqdm(total=len(os.listdir('./Dataset/test/'))) as pbar:
    for i in os.listdir('./Dataset/test/'):
        img = cv2.imread(f'./Dataset/test/{i}')
        test_y.append(i.split('.')[0].split('_')[1])
        test_X.append(extract_features(img))
        pbar.update(1)
test_X = np.array(test_X)
test_y = np.array(test_y)

100%|████████████████████████████████████████████████████████████████████████████| 40000/40000 [24:39<00:00, 27.03it/s]


In [11]:
scaler = StandardScaler()
train_X = scaler.fit_transform(train_X)
test_X = scaler.transform(test_X)

In [12]:
import joblib

In [13]:
joblib.dump(scaler, './scaler.pkl')

['./scaler.pkl']

In [14]:
np.save('./trainX.npy', train_X)
np.save('./testX.npy', test_X)
np.save('./trainy.npy', train_y)
np.save('./testy.npy', test_y)